Version: 0.0.2  Updated date: 07/05/2024
Conda Environment : py-snowpark_df_ml_fs-1.15.0_v1

# Getting Started with Snowflake Feature Store -  Customer Segmentation

The Customer segmentation (UC01) use case is designed to emulate a data science pipeline to find clusters of customers based  on  aggregate  features where  the  customers  are  grouped  based  on  their  spending behavior. <br>

It  involves  creating subgroups of customers based on similar traits. <br>

The input in this use case consists of order and return transaction data from a retail business. <br>

The use case uses Tables Customer, Order, Lineitem and Order_returns. <br>

K-means  clustering  algorithm  is  used  to  derive  the  optimum  number  of  clusters  and  understand  the  underlying customer segments based on the data provided. <br>
Clustering is an unsupervised machine learning technique, where there are no defined dependent and independent variables, i.e. the training samples are unlabeled. <br>
The pattern in the data is used to identify and group similar observations. <br>

We will use the Use-Case to show how Snowflake Feature Store (and Model Registry) can be used to maintain & store features, retrieve them for training and perform micro-batch inference.

In the development (TRAINING) enviroment we will 
- create FeatureViews in the Feature Store that maintain the required customer-behaviour features.
- use these Features to train a model, and save the model in the Snowflake model-registry.
- plot the clusters for the trained model to visually verify. 

In the production (SERVING) environment we will
- re-create the FeatureViews on production data
- generate an Inference FeatureView that uses the saved model to perform incremental inference

# Model Operationalisation in Production

In [1]:
%load_ext autoreload
%autoreload 2

#### Notebook Packages

In [2]:
# Python packages
import os
import json

# SNOWFLAKE
# Snowpark
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec
#from snowflake.snowpark import Analytics

import snowflake.snowpark.functions as F

# Snowflake Feature Store
from snowflake.ml.feature_store import (
    FeatureView,
    Entity)

# COMMON FUNCTIONS
from useful_fns import check_and_update, formatSQL, create_ModelRegistry, create_FeatureStore, create_SF_Session

# Feature Engineering Functions
from feature_engineering_fns import uc01_load_data, uc01_pre_process

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/opt/miniconda3/envs/py-snowpark_df_ml_fs/lib/python3.10/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/opt/miniconda3/envs/py-snowpark_df_ml_fs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Snowflake connection and database parameters

We point the `tpcxai_schema` variable to our `SERVING` schema, and this one change allows us to recreate the model development pipeline in production.

In [3]:
# Set the Schema (Environment)
tpcxai_schema = 'SERVING'

In [4]:
fs_qs_role, tpcxai_database, tpcxai_training_schema, session, warehouse_env = create_SF_Session(tpcxai_schema)

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.



Connection Established with the following parameters:
User                        : JARCHEN
Role                        : "FS_QS_ROLE"
Database                    : "TPCXAI_SF0001_QUICKSTART_INC"
Schema                      : "SERVING"
Warehouse                   : "TPCXAI_SF0001_QUICKSTART_WH"
Snowflake version           : 9.23.1
Snowpark for Python version : 1.35.0 



### MODEL OPERATIONALISATION
* Recreate production Entity, FeatureViews in Production FeatureStore
* Reuse the model fitted in development/training
* Create new Inference FeatureView for incremental model-inference

#### Setup Production Feature Store and references

In [ ]:
# Create/Reference Snowflake Model Registry - Common across Environments
mr = create_ModelRegistry(session, tpcxai_database, '_MODEL_REGISTRY')

# Create/Reference Snowflake Feature Store for Training (Development) Environment
fs = create_FeatureStore(session, tpcxai_database, f'''_{tpcxai_schema}_FEATURE_STORE''', warehouse_env)

### Reference Data to Snowflake Dataframe Objects
# Tables
line_item_tbl              = '.'.join([tpcxai_database, tpcxai_schema,'LINEITEM'])
order_tbl                  = '.'.join([tpcxai_database, tpcxai_schema,'ORDERS'])
order_returns_tbl          = '.'.join([tpcxai_database, tpcxai_schema,'ORDER_RETURNS'])

# Snowpark Dataframe
line_item_sdf              = session.table(line_item_tbl)
order_sdf                  = session.table(order_tbl)
order_returns_sdf          = session.table(order_returns_tbl)
print('''--- Created Data References ---''')

# Model Name
model_name = "UC01_SNOWFLAKEML_KMEANS_MODEL"


Model Registry (_MODEL_REGISTRY) already exists
Feature Store (_SERVING_FEATURE_STORE) already exists
--- Created Data References ---


We can now rerun the exact same code that we lifted from our Development (TRAINING) process to recreate the Feature Engineering pipelines in production

In [6]:
### CUSTOMER Entity
if "CUSTOMER" not in json.loads(fs.list_entities().select(F.to_json(F.array_agg("NAME", True))).collect()[0][0]):
    customer_entity = Entity(name="CUSTOMER", join_keys=["O_CUSTOMER_SK"],desc="Primary Key for CUSTOMER")
    fs.register_entity(customer_entity)
else:
    customer_entity = fs.get_entity("CUSTOMER")
print('''--- Created CUSTOMER Entity ---''')

### Create & Load Source Data
raw_data = uc01_load_data(order_sdf, line_item_sdf, order_returns_sdf)
rd_sql = formatSQL(raw_data.queries['queries'][0], True)
print('''--- Created Source Data ---''')

### Create & Run Preprocessing Function 
preprocessed_data = uc01_pre_process(raw_data)
ppd_sql = formatSQL(preprocessed_data.queries['queries'][0], True)
print('''--- Created Preprocessed Data ---''')

### Create Preprocessing FeatureView from Preprocess Dataframe (SQL)
ppd_fv_name = "FV_UC01_PREPROCESS"
ppd_fv_version = "V_1"
# Define descriptions for the FeatureView's Features.  These will be added as comments to the database object
preprocess_features_desc = { "FREQUENCY":"Average yearly order frequency",
                             "RETURN_RATIO":"Average of, Per Order Returns Ratio.  Per order returns ratio : total returns value / total order value" }
# Create Inference Feature View
try:
    # If FeatureView already exists just return the reference to it
    fv_uc01_preprocess = fs.get_feature_view(name=ppd_fv_name,version=ppd_fv_version)
except:
    # Create the FeatureView instance
    fv_uc01_preprocess_instance = FeatureView(
        name=ppd_fv_name, 
        entities=[customer_entity], 
        #feature_df=preprocessed_data,      # <- We can use the snowpark dataframe as-is from our Python
        feature_df=session.sql(ppd_sql),    # <- Or we can use SQL, in this case linted from the dataframe generated SQL to make more human readable
        timestamp_col="LATEST_ORDER_DATE",
        refresh_freq="60 minute",           # <- specifying optional refresh_freq creates FeatureView as Dynamic Table, else created as View.
        desc="Features to support Use Case 01").attach_feature_desc(preprocess_features_desc)

    # Register the FeatureView instance.  Creates  object in Snowflake
    fv_uc01_preprocess = fs.register_feature_view(
        feature_view=fv_uc01_preprocess_instance, 
        version=ppd_fv_version, 
        block=True
    )
    print(f"Feature View : {ppd_fv_name}_{ppd_fv_version} created in {tpcxai_schema}")   
else:
    print(f"Feature View : {ppd_fv_name}_{ppd_fv_version} already created in {tpcxai_schema}")

print('''---            DONE               ---''')


--- Created CUSTOMER Entity ---
--- Created Source Data ---
--- Created Preprocessed Data ---
Feature View : FV_UC01_PREPROCESS_V_1 already created in SERVING
---            DONE               ---


#### Create Scheduled Inference Pipeline

We now recreate our model inference process that will
- retrieve the latest version of the model from the Model Registry.
- read features from our feature pipeline (fv_uc01_preprocess featureview)
- pass features & model into inference function (uc01_serve) and return inference dataframe
- use inference dataframe to define a new FeatureView to maintain inference process

In [7]:
# Create an Inference Dataframe that reads from our feature-engineering pipeline
inference_input_sdf = fs.read_feature_view(fv_uc01_preprocess)
inference_input_sdf.show()

------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"FREQUENCY"  |"RETURN_RATIO"  |"LATEST_ORDER_DATE"  |
------------------------------------------------------------------------
|0                |2.5          |0.739528316     |2025-05-28           |
|1                |4.0          |0.7729305725    |2025-07-24           |
|2                |2.5          |0.757613162     |2025-06-26           |
|3                |2.0          |0.7503343175    |2025-07-24           |
|4                |2.0          |0.60744947      |2024-09-06           |
|6                |1.0          |0.59506324      |2024-07-04           |
|7                |2.5          |0.693000318     |2025-07-01           |
|8                |7.0          |0.733080465714  |2025-07-14           |
|9                |1.5          |0.95336742      |2025-02-11           |
|10               |2.0          |0.978573585     |2025-08-14           |
---------------------------------------------------

In [8]:
# Get latest version of the model
m = mr.get_model(model_name)
latest_version = m.show_versions().iloc[-1]['name']
mv = m.version(latest_version)

In [9]:
def uc01_serve(featurevector, km4_purchases) -> DataFrame:
    return km4_purchases.run(featurevector, function_name="predict")

# Test Inference process
inference_result_sdf = uc01_serve(inference_input_sdf, mv)
inference_result_sdf.sort(F.col('LATEST_ORDER_DATE').desc(), F.col('O_CUSTOMER_SK')).show()

----------------------------------------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"LATEST_ORDER_DATE"  |"RETURN_RATIO"  |"FREQUENCY"  |"RETURN_RATIO_MMS"   |"FREQUENCY_MMS"  |"CLUSTER"  |
----------------------------------------------------------------------------------------------------------------------------
|127              |2025-08-18           |0.930791813333  |3.0          |0.4123307841728324   |0.25             |0          |
|217              |2025-08-18           |1.0             |1.0          |0.451379751984621    |0.0              |1          |
|561              |2025-08-18           |0.694656926667  |1.5          |0.2790976504537308   |0.0625           |3          |
|645              |2025-08-18           |0.92235543      |3.0          |0.40757076838447914  |0.25             |0          |
|683              |2025-08-18           |0.87059023      |1.5          |0.37836356435388097  |0.0625           |1          |


We can see in the SQL output below how our model is packaged and called from SQL `MODEL_VERSION_ALIAS!PREDICT(RETURN_RATIO, FREQUENCY) AS TMP_RESULT`

In [10]:
ind_sql = inference_result_sdf.queries['queries'][0]
ind_fmtd_sql = os.linesep.join(ind_sql.split(os.linesep)[:1000])
print(ind_fmtd_sql)

SELECT 
    "O_CUSTOMER_SK", 
    "LATEST_ORDER_DATE", 
     CAST ("TMP_RESULT_J7H548AFIE"['RETURN_RATIO'] AS DOUBLE) AS "RETURN_RATIO", 
     CAST ("TMP_RESULT_J7H548AFIE"['FREQUENCY'] AS DOUBLE) AS "FREQUENCY", 
     CAST ("TMP_RESULT_J7H548AFIE"['RETURN_RATIO_MMS'] AS DOUBLE) AS "RETURN_RATIO_MMS", 
     CAST ("TMP_RESULT_J7H548AFIE"['FREQUENCY_MMS'] AS DOUBLE) AS "FREQUENCY_MMS", 
     CAST ("TMP_RESULT_J7H548AFIE"['CLUSTER'] AS BIGINT) AS "CLUSTER"
 FROM (
WITH SNOWPARK_ML_MODEL_INFERENCE_INPUT_AX1T81X4SW AS (SELECT * FROM TPCXAI_SF0001_QUICKSTART_INC._SERVING_FEATURE_STORE.FV_UC01_PREPROCESS$V_1),MODEL_VERSION_ALIAS_15V62D4YGK AS MODEL TPCXAI_SF0001_QUICKSTART_INC._MODEL_REGISTRY.UC01_SNOWFLAKEML_KMEANS_MODEL VERSION V_2
                SELECT *,
                    MODEL_VERSION_ALIAS_15V62D4YGK!PREDICT(RETURN_RATIO, FREQUENCY) AS TMP_RESULT_J7H548AFIE
                FROM SNOWPARK_ML_MODEL_INFERENCE_INPUT_AX1T81X4SW
)


### Create & Register Inference-FeatureView to run scheduled Inference

We can now define a new Inference Feature View using our Spine and Dataframe reading from our Feature Engineering pipeline.  The FeatureView when created as a Dynamic Table will run to the required refresh_freq and automatically perform incremental inference on new data that arrives through the pipeline.

In [11]:
## Create & Register Inference-FeatureView to run scheduled Inference
inf_fvname = "FV_UC01_INFERENCE_RESULT"
inf_fv_version = "V_2"

inference_features_desc = { "FREQUENCY":"Average yearly order frequency",
                              "RETURN_RATIO":"Average of, Per Order Returns Ratio.  Per order returns ratio : total returns value / total order value", 
                              "RETURN_RATIO_MMS":f"Min/Max Scaled version of RETURN_RATIO using Model Registry ({tpcxai_database}_MODEL_REGISTRY) Model ({mv.model_name}) Model-Version({mv.version_name}) Model Comment ({mv.comment})",
                              "FREQUENCY_MMS":f"Min/Max Scaled version of FREQUENCY using Model Registry ({tpcxai_database}_MODEL_REGISTRY) Model ({mv.model_name}) Model-Version({mv.version_name})  Model Comment ({mv.comment}",
                              "CLUSTER":f"Kmeans Cluster for Customer Clustering Model (UC01) using Model Registry ({tpcxai_database}_MODEL_REGISTRY) Model ({mv.model_name}) Model-Version({mv.version_name})  Model Comment ({mv.comment}"}

try:
   fv_uc01_inference_result = fs.get_feature_view(name= inf_fvname, version= inf_fv_version)
except:
   fv_uc01_inference_result = FeatureView(
         name= inf_fvname, 
         entities=[customer_entity], 
         feature_df=inference_result_sdf,
         ## refresh_freq="60 minute",
         desc="Inference Result from kmeans model for Use Case 01").attach_feature_desc(inference_features_desc)
   
   fv_uc01_inference_result = fs.register_feature_view(
         feature_view=fv_uc01_inference_result, 
         version= inf_fv_version, 
         block=True
   )
   print(f"Inference Feature View : fv_uc01_inference_result_{inf_fv_version} created")   
else:
   print(f"Inference Feature View : fv_uc01_inference_result_{inf_fv_version} already created")
finally:
   fs_serving_fviews = fs.list_feature_views().filter(F.col("NAME") == inf_fvname ).sort(F.col("VERSION").desc())
   fs_serving_fviews.show()  

Inference Feature View : fv_uc01_inference_result_V_2 already created
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"NAME"                    |"VERSION"  |"DATABASE_NAME"               |"SCHEMA_NAME"           |"CREATED_ON"                |"OWNER"     |"DESC"                                              |"ENTITIES"    |"REFRESH_FREQ"  |"REFRESH_MODE"  |"SCHEDULING_STATE"  |"WAREHOUSE"  |"CLUSTER_BY"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|FV_UC01_INFERENCE_RESULT  |V_2        |TPCXAI_SF0001_QUI

In [12]:
fv_uc01_inference_result

FeatureView(_name=FV_UC01_INFERENCE_RESULT, _entities=[Entity(name=CUSTOMER, join_keys=['O_CUSTOMER_SK'], owner=None, desc=Primary Key for CUSTOMER)], _feature_df=<snowflake.snowpark.dataframe.DataFrame object at 0x153c808e0>, _timestamp_col=None, _desc=Inference Result from kmeans model for Use Case 01, _infer_schema_df=<snowflake.snowpark.dataframe.DataFrame object at 0x153c820e0>, _query=SELECT 
    "O_CUSTOMER_SK", 
    "LATEST_ORDER_DATE", 
     CAST ("TMP_RESULT_CAZ3PHFWYS"['RETURN_RATIO'] AS DOUBLE) AS "RETURN_RATIO", 
     CAST ("TMP_RESULT_CAZ3PHFWYS"['FREQUENCY'] AS DOUBLE) AS "FREQUENCY", 
     CAST ("TMP_RESULT_CAZ3PHFWYS"['RETURN_RATIO_MMS'] AS DOUBLE) AS "RETURN_RATIO_MMS", 
     CAST ("TMP_RESULT_CAZ3PHFWYS"['FREQUENCY_MMS'] AS DOUBLE) AS "FREQUENCY_MMS", 
     CAST ("TMP_RESULT_CAZ3PHFWYS"['CLUSTER'] AS BIGINT) AS "CLUSTER"
 FROM (
WITH SNOWPARK_ML_MODEL_INFERENCE_INPUT_9XSC80FPEO AS (SELECT * FROM TPCXAI_SF0001_QUICKSTART_INC._SERVING_FEATURE_STORE.FV_UC01_PREPROCESS$V

In [13]:
fv_uc01_inference_result.feature_df.sort(F.col("LATEST_ORDER_DATE").desc()).show(100)

----------------------------------------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"LATEST_ORDER_DATE"  |"RETURN_RATIO"  |"FREQUENCY"  |"RETURN_RATIO_MMS"   |"FREQUENCY_MMS"  |"CLUSTER"  |
----------------------------------------------------------------------------------------------------------------------------
|744              |2025-08-18           |0.783372975     |2.0          |0.32915343550720966  |0.125            |0          |
|2847             |2025-08-18           |0.75            |2.0          |0.31032357896395174  |0.125            |0          |
|4015             |2025-08-18           |0.711780404     |2.5          |0.28875913917932744  |0.1875           |0          |
|2714             |2025-08-18           |0.754647276     |2.5          |0.312945686834075    |0.1875           |0          |
|3602             |2025-08-18           |0.78373749      |1.5          |0.32935910387084416  |0.0625           |2          |


## CLEAN UP

In [14]:
session.close()